In [1]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),    
)

In [3]:
# Define the batchs
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y {y.shape} {y.dtype}")
    break    

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64]) torch.int64


In [4]:
# Move operations to GPU or MPS if available
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using mps device


In [5]:
# Define the model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
# makes prediction, backpropagates the prediction error and optimize
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")

In [8]:
# check the model performance against the test dataset
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5

for t in range(epochs):
    print()
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


loss: 2.311682 [   64 / 60000]
loss: 2.295432 [ 6464 / 60000]
loss: 2.280793 [12864 / 60000]
loss: 2.274358 [19264 / 60000]
loss: 2.245910 [25664 / 60000]
loss: 2.223042 [32064 / 60000]
loss: 2.227989 [38464 / 60000]
loss: 2.198778 [44864 / 60000]
loss: 2.192380 [51264 / 60000]
loss: 2.153187 [57664 / 60000]
Test Error: 
 Accuracy: 49.4%, Avg loss: 2.154489 


loss: 2.166291 [   64 / 60000]
loss: 2.154888 [ 6464 / 60000]
loss: 2.102006 [12864 / 60000]
loss: 2.117384 [19264 / 60000]
loss: 2.058970 [25664 / 60000]
loss: 2.006436 [32064 / 60000]
loss: 2.025249 [38464 / 60000]
loss: 1.951488 [44864 / 60000]
loss: 1.956007 [51264 / 60000]
loss: 1.876748 [57664 / 60000]
Test Error: 
 Accuracy: 56.4%, Avg loss: 1.877478 


loss: 1.914342 [   64 / 60000]
loss: 1.882054 [ 6464 / 60000]
loss: 1.766135 [12864 / 60000]
loss: 1.807039 [19264 / 60000]
loss: 1.695079 [25664 / 60000]
loss: 1.651936 [32064 / 60000]
loss: 1.667157 [38464 / 60000]
loss: 1.570357 [44864 / 60000]
loss: 1.597225 [51264 / 6

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [15]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
